In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas_profiling
import time

from sklearn.preprocessing import LabelBinarizer
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [6]:
CATEGORICAL_COLUMNS = ["bldg_ctgy", "bldg_cls_s", "bldg_cls_p", "tax_cls_p", "tax_cls_s"]
LABEL_COLUMN = "price"
LABEL_THRESHOLD = 500000
SECOND_TEST_COLUMNS = ["price", "bldg_cls_s", "borough", "lot", "tot_sqft"]
FIRST_TEST_COLUMNS = [*SECOND_TEST_COLUMNS, "long", "lat", "block", "lot", "yr_built", "tot_unit", "bldg_ctgy"]

In [7]:
def encode_column(data, column):
    encoded_columns = pd.get_dummies(data[column], prefix=column)
    return pd.concat([data, encoded_columns], axis=1).drop(column, axis=1)

In [8]:
# raw_data = pd.read_csv("sales_data_2015.csv")
# pd.set_option("display.max_columns", None)

# # Some pre-processing
# data = raw_data[(raw_data.usable == 'Y') & 
#                 (raw_data.price > 1000) &
#                 (raw_data.tot_sqft > 0) &
#                 (raw_data.yr_built > 0) &
#                 (raw_data.tot_unit > 0)]
# data = data.drop(columns=["Unnamed: 0", "year", "apt", "easmnt", "usable", "address", "bbl_id", "Sale_id", "sale_date"])
# data.isna().sum().sort_values(ascending=False)

# features = {}
# features[2] = list(data.drop(LABEL_COLUMN, axis=1).columns)
# features[0] = [f for f in features[2] if f not in FIRST_TEST_COLUMNS]
# features[1] = [f for f in features[2] if f not in SECOND_TEST_COLUMNS]

# for category in CATEGORICAL_COLUMNS:
#     data = encode_column(data, category)

# display(data)
# data.to_csv("ny_housing.csv")

In [9]:
data = pd.read_csv("ny_housing.csv")

features = {}
features[2] = list(data.drop(LABEL_COLUMN, axis=1).columns)
features[0] = [f for f in features[2] if f not in FIRST_TEST_COLUMNS]
features[1] = [f for f in features[2] if f not in SECOND_TEST_COLUMNS]

In [10]:
data["price_class"] = np.where(data["price"] > 500000, 1, 0)
display(data)

,Unnamed: 0,borough,block,lot,zip,res_unit,com_unit,tot_unit,land_sqft,tot_sqft,...,tax_cls_p_1D,tax_cls_p_2,tax_cls_p_2A,tax_cls_p_2B,tax_cls_p_2C,tax_cls_p_4,tax_cls_s_1,tax_cls_s_2,tax_cls_s_4,price_class
0,4,2,3039,36,10458,1,1,2,2500,2452,...,0,0,0,0,0,0,1,0,0,0
1,7,2,2912,139,10457,2,0,2,2000,2400,...,0,0,0,0,0,0,1,0,0,0
2,8,2,2912,152,10457,2,0,2,2000,2400,...,0,0,0,0,0,0,1,0,0,0
3,9,2,2929,126,10457,2,0,2,3165,2394,...,0,0,0,0,0,0,1,0,0,1
4,10,2,3030,151,10457,2,1,3,1620,3240,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27881,84758,5,7106,10,10309,2,0,2,3447,1200,...,0,0,0,0,0,0,1,0,0,0
27882,84759,5,7267,91,10309,2,0,2,5645,1550,...,0,0,0,0,0,0,1,0,0,1
27883,84761,5,7267,145,10309,2,0,2,5700,1600,...,0,0,0,0,0,0,1,0,0,0
27884,84763,5,7316,16,10309,2,0,2,3180,1300,...,0,0,0,0,0,0,1,0,0,0


In [11]:
records = []

for train_size in [0.2, 0.5, 0.8]:

    train, test = train_test_split(data, test_size=1 - train_size)
    print(len(train), 'train examples')
    print(len(test), 'test examples')
    
    for features_version in [0, 1, 2]:
        
        print("Train size: ", train_size, "Features version: ", features_version)
        
        columns_X = [col for col in data if any([col.startswith(feature) for feature in features[features_version]])]
        columns_Y = [LABEL_COLUMN]
        
        train_ds = train[[*columns_X, *columns_Y, "price_class"]]
        test_ds = test[[*columns_X, *columns_Y, "price_class"]]
        
        train_X = train_ds[columns_X]
        train_Y = train_ds[columns_Y]
        test_X = test_ds[columns_X]
        test_Y = test_ds[columns_Y]
        
        time_before = time.perf_counter()
        model = LinearRegression().fit(train_X, train_Y)
        time_taken = time.perf_counter() - time_before
        
        train_score = accuracy_score(train_ds["price_class"], np.where(model.predict(train_X) > 500000, 1, 0))
        test_score = accuracy_score(test_ds["price_class"], np.where(model.predict(test_X) > 500000, 1, 0))
        
        records.append([time_taken, train_score, test_score])

5577 train examples
22309 test examples
Train size:  0.2 Features version:  0
Train size:  0.2 Features version:  1
Train size:  0.2 Features version:  2
13943 train examples
13943 test examples
Train size:  0.5 Features version:  0
Train size:  0.5 Features version:  1
Train size:  0.5 Features version:  2
22308 train examples
5578 test examples
Train size:  0.8 Features version:  0
Train size:  0.8 Features version:  1
Train size:  0.8 Features version:  2


In [17]:
results = []
i = 0
for train_size in [0.2, 0.5, 0.8]:
    for features_version in [0, 1, 2]:
        results.append({
            'split': train_size, 
            'features': features_version,
            'time': round(records[i][0], 3) ,
            'train_acc': round(records[i][1], 3),
            'test_acc': round(records[i][2], 3)
        })
        
        i = i+1

In [18]:
results_df = pd.DataFrame(results)
results_df

,split,features,time,train_acc,test_acc
0,0.2,0,0.050,0.538,0.531
1,0.2,1,0.053,0.506,0.506
2,0.2,2,0.052,0.551,0.542
3,0.5,0,0.110,0.575,0.579
4,0.5,1,0.118,0.610,0.615
5,0.5,2,0.116,0.603,0.600
6,0.8,0,0.158,0.579,0.578
7,0.8,1,0.233,0.578,0.581
8,0.8,2,0.187,0.571,0.576


In [19]:
results_df.to_csv('LR_ny_housing_result.csv')

In [20]:
print(results_df.to_latex())

\begin{tabular}{lrrrrr}
\toprule
{} &  split &  features &   time &  train\_acc &  test\_acc \\
\midrule
0 &    0.2 &         0 &  0.050 &      0.538 &     0.531 \\
1 &    0.2 &         1 &  0.053 &      0.506 &     0.506 \\
2 &    0.2 &         2 &  0.052 &      0.551 &     0.542 \\
3 &    0.5 &         0 &  0.110 &      0.575 &     0.579 \\
4 &    0.5 &         1 &  0.118 &      0.610 &     0.615 \\
5 &    0.5 &         2 &  0.116 &      0.603 &     0.600 \\
6 &    0.8 &         0 &  0.158 &      0.579 &     0.578 \\
7 &    0.8 &         1 &  0.233 &      0.578 &     0.581 \\
8 &    0.8 &         2 &  0.187 &      0.571 &     0.576 \\
\bottomrule
\end{tabular}

